# plot_2d examples
**Author: Jun Sasaki Coded on 2025-05-07 Updated on 2025-05-10**<br>

In [ ]:
from pathlib import Path
import cartopy.crs as ccrs
from cartopy.io.img_tiles import GoogleTiles
from xfvcom import FvcomDataLoader
from xfvcom import FvcomPlotter, FvcomPlotConfig, FvcomPlotOptions

from IPython.core.magic import register_cell_magic
@register_cell_magic
def skip(line, cell):
    print("This cell is skipped.")

png_dir = Path("PNG"); png_dir.mkdir(exist_ok=True)

## Create FvcomDataLoader, FvcomPlotConfig, and FvcomPlotter instances.
- Specify an FVCOM output netcdf path (`ncfile_path`) and input open boundary file path (`obcfile_path`).
- Suppose `fvcom` is the FvcomDataLoader instance, Dataset is `fvcom.ds`.

In [ ]:
base_path = Path("~/Github/TB-FVCOM/goto2023").expanduser()
ncfile = "TokyoBay18_r16_crossed_0001.nc"
obcfile = "TokyoBay18_obc.dat"
ncfile_path = base_path / "output" / ncfile
obcfile_path = base_path / "input" / obcfile
fvcom = FvcomDataLoader(ncfile_path=ncfile_path, obcfile_path=obcfile_path, time_tolerance=5)
cfg = FvcomPlotConfig()
plotter = FvcomPlotter(fvcom.ds, cfg)

## Mesh and map tiles only

In [ ]:
opts = FvcomPlotOptions(
    figsize=(4,6),
    add_tiles=True,
    tile_provider=GoogleTiles(style="satellite"),
    mesh_color="#ffffff",
    mesh_linewidth=0.3,
    xlim = ("139:40:00", "140:00:00"),
    ylim = ("35:12:00", "35:42"),
    lon_tick_skip=2,
)
plotter.plot_2d(da=None, opts=opts)

## Scalar + vector overlay at a `time` and `siglay`
### Pass index selected scalar and vector DataArrays
This approach may avoid file access conflicts and efficiently create animation frames using multiprocessing.

In [ ]:
var_scalar = 'temp'
time = 20; siglay=0
da_s = fvcom.ds[var_scalar].isel(time=time, siglay=siglay)
da_u = fvcom.ds['u'].isel(time=time, siglay=siglay)
da_v = fvcom.ds['v'].isel(time=time, siglay=siglay)

opts = FvcomPlotOptions(
    figsize=(4,6),
    plot_vec2d=True,
    with_mesh=False,
    coastlines=True,
    lon_tick_skip=2,
)
plotter.plot_2d(da=da_s, da_u=da_u, da_v=da_v, opts=opts)

### The same plot but pass DataArrays without selecting index and save it as PNG
The selected index is specified in FvcomPlotOptions. The vector element varnames can be specified with `var_u` and `var_v`.

In [ ]:
var_scalar = 'temp'
time = 20; siglay=0
da_s = fvcom.ds[var_scalar]

opts = FvcomPlotOptions(
    figsize=(4,6),
    scalar_time = time,
    scalar_siglay = siglay,
    plot_vec2d=True,
    vec_time = time,
    vec_siglay = siglay,
    with_mesh=False,
    coastlines=True,
    lon_tick_skip=2,
)
ax = plotter.plot_2d(da=da_s, var_u='u', var_v='v', opts=opts)
ax.figure.savefig(png_dir / "scalar_vector.png", dpi=600, bbox_inches='tight')

### The same plot decorating with text using `post_process_func`

In [ ]:
var_scalar = 'temp'
time = 20; siglay=0
da_s = fvcom.ds[var_scalar]

def add_timestamp(ax, da, time):
    import pandas as pd
    txt = pd.to_datetime(time).strftime("%Y-%m-%d %H:%M")
    ax.text(
        0.95, 0.05, txt,
        transform=ax.transAxes,
        ha="right", va="top",
        color="white",
        fontsize=10,
        bbox=dict(fc="0.2", ec="none", alpha=0.7, pad=2),
    )

opts = FvcomPlotOptions(
    figsize=(4,6),
    scalar_time = time,
    scalar_siglay = siglay,
    plot_vec2d=True,
    vec_time = time,
    vec_siglay = siglay,
    with_mesh=False,
    coastlines=True,
    lon_tick_skip=2,
)
plotter.plot_2d(da=da_s, var_u='u', var_v='v', opts=opts, post_process_func=add_timestamp)

## Scalar + vector overlay with time/vertical averaging

In [ ]:
var_scalar = 'temp'
da_s = fvcom.ds[var_scalar]
scalar_time = slice("2020-01-03", "2020-01-04")
scalar_siglay = slice(None)
scalar_reduce = {"time": "mean", "siglay": "mean"}
vec_time      = slice("2020-01-03", "2020-01-04")
vec_siglay    = slice(None)
vec_reduce    = {"time": "mean", "siglay": "mean"}

opts = FvcomPlotOptions(
    figsize=(4,6),
    scalar_time = scalar_time,
    scalar_siglay = scalar_siglay,
    scalar_reduce = scalar_reduce,
    plot_vec2d=True,
    vec_time = vec_time,
    vec_siglay = vec_siglay,
    vec_reduce = vec_reduce,
    with_mesh=False,
    coastlines=True,
    lon_tick_skip=2,
)
plotter.plot_2d(da=da_s, var_u='u', var_v='v', opts=opts)

In [ ]:
var_scalar = 'temp'
da_s = fvcom.ds[var_scalar]
scalar_time = 20
scalar_siglay = slice(None)
scalar_reduce = {"time": "mean", "siglay": "mean"}
vec_time      = 20
vec_siglay    = slice(None)
vec_reduce    = {"time": "mean", "siglay": "mean"}

opts = FvcomPlotOptions(
    figsize=(4,6),
    scalar_time = scalar_time,
    scalar_siglay = scalar_siglay,
    scalar_reduce = scalar_reduce,
    plot_vec2d=True,
    vec_time = vec_time,
    vec_siglay = vec_siglay,
    vec_reduce = vec_reduce,
    with_mesh=False,
    coastlines=True,
    lon_tick_skip=2,
)
plotter.plot_2d(da=da_s, var_u='u', var_v='v', opts=opts)